In [5]:
from collections import defaultdict
import numpy as np
from itertools import combinations
from dwave.system.samplers import DWaveSampler
from dwave.system import LeapHybridSampler
from dwave.system.composites import EmbeddingComposite
from matplotlib import pyplot as plt
import networkx as nx
import networkx.generators.community as community
from networkx.generators.random_graphs import connected_watts_strogatz_graph as small_world
from networkx.generators.random_graphs import powerlaw_cluster_graph as cluster
import ndlib
import future.utils
import warnings
import time
from random import randint
from random import sample
warnings.simplefilter('ignore')
%matplotlib inline

In [23]:
# Create QUBO dict for D-Wave to optimize
def maxcut_qubo(G):
    # The Hamiltonian is H = sum(Pauli_Z(i) x Pauli_Z(j) - I)/2
    # The eigenvalue is the negation of the edges in cut solution
    # The QUBO should be -xi-xj+2xixj, we label it as Q
    Q = defaultdict(int)
    for u, v in G.edges:
        Q[(u,u)]+= -1
        Q[(v,v)]+= -1
        Q[(u,v)]+= 2
    return Q

# Invoke D-Wave annealer
def solve(Q, chainstrength=8, numruns=100):
    # Run the QUBO on the solver from your config file
    sampler = LeapHybridSampler()
    response = sampler.sample_qubo(Q, num_reads=numruns)
    energies = iter(response.data())
    return response

In [8]:
def random_avg_distance(G, samples):
    distance_sum = 0
    for i in range(samples):
        pair = sample(list(G.nodes()), 2)
        #print(pair)
        distance = len(nx.shortest_path(G, pair[0], pair[1]))
        #print(distance)
        distance_sum += distance
    return distance_sum/samples

def random_avg_clustering_coeff(G, samples):
    sampled_nodes = sample(list(G.nodes()), samples)
    clustering_dict = nx.clustering(G, sampled_nodes)
    return np.mean(np.array(list(clustering_dict.values())))

In [9]:
densities, cluster_coeffs, avg_distances = [], [], []
a = small_world(100, 4, .05)
densities.append(nx.density(a))
cluster_coeffs.append(random_avg_clustering_coeff(a, 50))
avg_distances.append(random_avg_distance(a, 50))
b = small_world(200, 8, .1)
densities.append(nx.density(b))
cluster_coeffs.append(random_avg_clustering_coeff(b, 50))
avg_distances.append(random_avg_distance(b, 50))
c = small_world(400, 16, .14)
densities.append(nx.density(c))
cluster_coeffs.append(random_avg_clustering_coeff(c, 50))
avg_distances.append(random_avg_distance(c, 50))
d = small_world(800, 32, .15)
densities.append(nx.density(d))
cluster_coeffs.append(random_avg_clustering_coeff(d, 50))
avg_distances.append(random_avg_distance(d, 50))
e = small_world(1600, 64, .15)
densities.append(nx.density(e))
cluster_coeffs.append(random_avg_clustering_coeff(e, 50))
avg_distances.append(random_avg_distance(e, 50))
f = small_world(3200, 128, .15)
densities.append(nx.density(f))
cluster_coeffs.append(random_avg_clustering_coeff(f, 50))
avg_distances.append(random_avg_distance(f, 50))
g = small_world(6400, 256, .15)
densities.append(nx.density(f))
cluster_coeffs.append(random_avg_clustering_coeff(f, 50))
avg_distances.append(random_avg_distance(f, 50))
h = small_world(12800, 512, .15)
densities.append(nx.density(f))
cluster_coeffs.append(random_avg_clustering_coeff(f, 50))
avg_distances.append(random_avg_distance(f, 50))
print(densities) 
print(cluster_coeffs) 
print(avg_distances)

[0.04040404040404041, 0.04020100502512563, 0.040100250626566414, 0.04005006257822278, 0.04002501563477173, 0.04001250390747108, 0.04001250390747108, 0.04001250390747108]
[0.4026666666666667, 0.48336507936507933, 0.4509361809501128, 0.4559357583175395, 0.46936473710331195, 0.452351081021374, 0.46249191141547513, 0.45652763727131296]
[6.7, 4.52, 3.96, 3.64, 3.32, 3.1, 3.08, 3.24]


In [30]:
small_world_graphs = [a, b, c, d, e, f, g, h]
small_world_times = []
small_world_start_time = time.time()
small_world_previous_time = small_world_start_time
for graph, size in zip(small_world_graphs, [100, 200, 400, 800, 1600, 3200, 6400, 12800]):
    result = solve(maxcut_qubo(graph))
    small_world_current_time = time.time()
    small_world_times.append(small_world_current_time - small_world_previous_time)
    small_world_previous_time = small_world_current_time
    print(f'completed size {size}')
print(small_world_times)

completed size 100
completed size 200
completed size 400
completed size 800
completed size 1600
completed size 3200
completed size 6400
completed size 12800
[1.7757821083068848, 1.9077961444854736, 1.6752736568450928, 1.9922738075256348, 2.7618930339813232, 6.263381481170654, 16.094152212142944, 90.20424818992615]


Using hybrid solvers, we can solve Max-cut on a sparse graph of 12 thousand nodes in 90 seconds

Contributors: James Sud